In [1]:
import pandas as pd

In [2]:
import os


In [3]:
pwd = os.getcwd()
head, tail = os.path.split(pwd)
listing_data = os.path.join(head, "data/raw/bkk_listings.csv")
df = pd.read_csv(listing_data)

In [4]:
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,27934,Nice room with superb city view,120437,Nuttee,NaN,Ratchathewi,13.75983,100.54134,Entire home/apt,1665.0,15,65,2024-09-17,0.41,1,362,1,NaN
1,27979,"Easy going landlord,easy place",120541,Emy,NaN,Bang Na,13.66818,100.61674,Private room,NaN,1,0,NaN,NaN,2,0,0,NaN
2,28745,modern-style apartment in Bangkok,123784,Familyroom,NaN,Bang Kapi,13.75232,100.62402,Private room,NaN,60,0,NaN,NaN,1,0,0,NaN
3,47516,Beautiful waterfront house,214456,Anuradha,NaN,Don Mueang,13.92726,100.58529,Entire home/apt,4373.0,3,0,NaN,NaN,1,364,0,NaN
4,48736,Condo with Chaopraya River View,222005,Athitaya,NaN,Rat Burana,13.68556,100.49535,Private room,1930.0,14,1,2014-02-03,0.01,1,365,0,NaN
